# Richmond Police Activity Maps

The maps below are obtained from Richmond Police Department and Wayne County media reports.

In [5]:
from __future__ import division, print_function
import pandas as pd
import os
import numpy as np
import folium
from folium import plugins


def read_input_data(include=None, exclude=None):
    '''Read the input data'''
    df = pd.read_csv('reports_all.csv')
    df = df.dropna(subset=['Latitude', 'Longitude'])
    if include:
        df = df[df.Details.str.contains('|'.join(include), case=False, na=False)]
    if exclude:
        df = df[~df.Details.str.contains('|'.join(exclude), case=False, na=False)]
    return df


def add_to_map(m, latitude, longitude, department, date=None, address=None, description=None):
    '''Add results to the map'''
    if department == 'richmond':
        color = 'red'
    else:
        color = 'blue'
    details = ''
    if date is not None:
        details += '<b>' + date + '</b><br>\n'
    if address is not None:
        details += '<i>' + address.replace(', Richmond, IN  47374', '') \
                                  .replace(', Wayne County, IN', '') + '</i><br>'
    if description is not None:
        details += description.replace("'", "\\'") + '\n'
    folium.Circle(location=[latitude, longitude],
                  popup=details,
                  radius=15,
                  color=color,
                  fill_color=color
                 ).add_to(m)


m = folium.Map(location=[39.8289, -84.8902],
                   tiles='Stamen Toner',
                   zoom_start=14,
                   detect_retina=True,
                   prefer_canvas=True)

df = read_input_data()

for index, row in df.iterrows():
    add_to_map(m, row['Latitude'], row['Longitude'], row['department'], 
               date=str(row['Date']), address=row['Address'],
               description=str(row['Details']))

heat_map_array = df[['Latitude', 'Longitude']].values
m.add_child(plugins.HeatMap(heat_map_array, radius=15))

m

Figure 1. Police activity map from the Richmond Police Department (red dots) and the Wayne County Sheriff's office (blue dots). More information about each event can be obtained by clicking on a dot.